<a href="https://colab.research.google.com/github/MinamiInoue2323/b4_train_day3/blob/main/BERT%E4%BA%8B%E5%89%8D%E5%AD%A6%E7%BF%92%E3%83%A2%E3%83%87%E3%83%AB%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%88%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考参考
https://note.com/npaka/n/n5bb043191cc9
https://qiita.com/karaage0703/items/30485c2ba1c396760982

# 環境構築

In [ ]:
#必要なライブラリのセットアップ
!pip install transformers
!pip install pandas
!pip install ipadic
!pip install fugashi

In [ ]:
#MeCabのインストール
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install fugashi ipadic

In [ ]:
#ドライブをcolaboratoryにマウント
from google.colab import drive
drive.mount('content/drive')

In [ ]:
#ライブラリが正しくインストールされているか確認、エラーが出なければエラーが出なければok
import torch
import MeCab

# マスク予測をやってみる

# 下準備

In [ ]:
#対象となる文章
sentence = '私は明日の午後近所の海鮮丼屋で[MASK]を食べる予定だ。'
input_sentence = '[CLS]{}[SEP]'.format(sentence)

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, BertConfig
import torch
from transformers import BertJapaneseTokenizer, pipeline, BertForMaskedLM, BertConfig

In [ ]:
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"

#modelの設定の定義
config = BertConfig.from_pretrained( model_name, output_hidden_states=True)

#tokenizerの読み込みと保存
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained("./tokenizer")
#tokenizerが正常に動くか確認
tokenizer.tokenize('お腹が痛いので遅れます。')

#モデルの読み込みと保存
model = BertForMaskedLM.from_pretrained(model_name, config = config)
model.save_pretrained('./model')

## 方法1:tokenizer

In [ ]:
##BERT入力前の下準備
#入力文をトークンに分割
tokenized_input = tokenizer.tokenize(input_sentence)
print(tokenized_input)

#トークン化された文章を文章をBERTに入力できる形に変換
id_input = tokenizer.convert_tokens_to_ids(tokenized_input)
print(id_input)
tensor_input = torch.tensor([id_input])
print(tensor_input)

#[MASK]のトークンのidを取得
mask_token_index = torch.where(tensor_input == tokenizer.mask_token_id)[1]

In [ ]:
show_top = 5 #予測の上位何件を表示するか
with torch.no_grad():
  token_logits = model(tensor_input).logits
  mask_token_logits = token_logits[0, mask_token_index, :]
  top_tokens = torch.topk(mask_token_logits, show_top, dim=1).indices[0].tolist()
  top_values = torch.topk(mask_token_logits, show_top, dim=1).values[0].tolist()
for i in range(show_top):
  token = tokenizer.decode(top_tokens[i])
  value = top_values[i]
  print('token: {}, confidence: {}'.format(token, value))


## 方法2:pipeline

In [ ]:
from transformers import pipeline

#パイプラインの作成
nlp = pipeline('fill-mask', model = model, tokenizer = tokenizer)

#mask predictionの実行
result = nlp(input_sentence)

#結果の出力
for r in result:
  print(r)